In [1]:
!git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 101 (delta 5), reused 10 (delta 3), pack-reused 81
Receiving objects: 100% (101/101), 25.27 KiB | 12.64 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [5]:
%cd ./py-hanspell
!python ./setup.py install

%cd ../

/opt/ml/workspace/level1_semantictextsimilarity-nlp-14/JS/py-hanspell
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
running install
running bdist_egg
running egg_info
writing py_hanspell.egg-info/PKG-INFO
writing dependency_links to py_hanspell.egg-info/dependency_links.txt
writing requirements to py_hanspell.egg-info/requires.txt
writing top-level names to py_hanspell.egg-info/top_level.txt
reading manifest file 'py_hanspell.egg-info/SOURCES.txt'
writing manifest file 'py_hanspell.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying hanspell/spell_checker.py -> build/lib/hanspell
copying hanspell/constants.py -> build/lib/hanspell
creating bui

In [7]:
%ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
09103base/                data_test/              output09105.csv
2023-04-12-002base_sota/  klue-roberta-small_JS/  output09147.csv
2023-04-16-001base/       large_long/             output_V3.csv
Readme.md                 lightning_logs/         py-hanspell/
STS_roberta_large/        load_09103/             save/
STS_roberta_large_002/    model.pt                spell_correction.ipynb
code/                     nohup.out               wandb/
data.ipynb                output09103.csv


In [2]:
# 기존 데이터세트에서 UNK로 인식되는 경우에는 스펠링 체크한 것으로 변경해주는 것이 우선이다(0417)
# 기존 데이터세트 UNK, 스펠링 체크진행
import re
import sys
import json

import pandas as pd
from matplotlib import pyplot as plt

from tqdm import tqdm
from transformers import AutoTokenizer
from collections import Counter, OrderedDict

sys.path.append('./py-hanspell')
from tqdm import tqdm
from hanspell import spell_checker


train_path = '~/data/train.csv'
dev_path = '~/data/dev.csv'

train_data = pd.read_csv(train_path)
dev_data = pd.read_csv(dev_path)



tokenizer = AutoTokenizer.from_pretrained('klue/roberta-small')

train_data = pd.read_csv(train_path)
train_data_unk_corrected = train_data.copy(deep=True)

for i, item in tqdm(train_data.iterrows()):
    sentence_1, sentence_2 = item['sentence_1'], item['sentence_2']
    unk1_tokens, unk2_tokens = [], []
    
    # 1-1. UNK check - sentene 1
    encode_1, encode_2 = tokenizer(sentence_1), tokenizer(sentence_2)
    if encode_1['input_ids'].count(tokenizer.unk_token_id):
        unk_tok_1 = []
        
        decode_1 = tokenizer.convert_ids_to_tokens(encode_1['input_ids'])
        unk_idx = [i for i, token in enumerate(decode_1) if token == tokenizer.unk_token]
        
        for _idx in unk_idx:
            char_index = encode_1.token_to_chars(_idx)
            original_token = sentence_1[char_index.start:char_index.end]  # char_index 는 CharSpan(start=15, end=19) 형태로 리턴되더랍니다... 신기!
            
            unk_tok_1.append(original_token)
        
        if unk_tok_1:
            unk1_tokens.append(unk_tok_1)
            
        # 2. spelling correction
        result_1 = spell_checker.check(sentence_1).as_dict()
        checked_1 = result_1['checked']
        
        if checked_1 != sentence_1:      # 하나라도 스펠링 바뀌면 바꾼 것으로 대체해서 넣는다.
            train_data_unk_corrected.loc[i, 'sentence_1'] = checked_1
        
    
    # 1-2. UNK check - sentene 2
    if encode_2['input_ids'].count(tokenizer.unk_token_id):
        unk_tok_2 = []
        
        decode_2 = tokenizer.convert_ids_to_tokens(encode_2['input_ids'])
        unk_idx = [i for i, token in enumerate(decode_2) if token == tokenizer.unk_token]
        
        for _idx in unk_idx:
            char_index = encode_2.token_to_chars(_idx)
            original_token = sentence_2[char_index.start:char_index.end]  # char_index 는 CharSpan(start=15, end=19) 형태로 리턴되더랍니다... 신기!
            
            unk_tok_2.append(original_token)
        
        if unk_tok_2:
            unk2_tokens.append(unk_tok_2)
            
        # 2. spelling correction
        result_2 = spell_checker.check(sentence_2).as_dict()
        checked_2 = result_2['checked']
        
        if checked_2 != sentence_2:      # 하나라도 스펠링 바뀌면 바꾼 것으로 대체해서 넣는다.
            train_data_unk_corrected.loc[i, 'sentence_2'] = checked_2
    
train_data_unk_corrected.to_csv("~/data/train_unk_corrected.csv")
train_data_unk_corrected.head(5)

9324it [00:22, 414.13it/s]


,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근 권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
